In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Skip because data is already merged
# iterate over all .csv files in directory and append them into one large file
#i=0
#for file in os.listdir(os.getcwd()):
 #   if file.endswith(".csv"):
  #      if i==0:
   #         train_data = pd.read_csv(file, sep='|')
    #        i+=1
     #   else:
      #      temp = pd.read_csv(file, sep='|')
       #     train_data = train_data.append(temp, ignore_index=True)        

In [2]:
train_data = pd.read_csv('TMDB_raw_data_merge.csv', encoding = 'utf8',  sep = '|', engine = 'python')
train_data.shape

(159212, 21)

In [3]:
# remove rows that have data in the wrong column or other idiosyncrasies 
train_data = train_data[train_data['id'].str.contains('~')==False]
train_data = train_data[train_data['budget'].str.contains('~')==False]
train_data = train_data[train_data['revenue'].str.contains('~')==False]
train_data = train_data[train_data['popularity'].str.contains('~')==False]
train_data = train_data[train_data['vote_count'].str.contains('~')==False]
train_data = train_data[train_data['vote_average'].str.contains('~')==False]
train_data = train_data[train_data['adult'].str.contains('~')==False]
train_data.shape

(156327, 21)

In [4]:
train_data = train_data.reset_index(drop=True)

In [5]:
# for each column that stores multiple values as a string separated by '~'
# split that string apart and store the name of that column to be broken into dummies
dummy_names = []
for name in list(train_data):
    if name != 'title' and name != 'original_title' and name != 'overview' and name != 'tagline':
        if train_data[name].dtype == 'O' and np.sum(train_data[name].str.contains('~'))!= 0 or name == 'original_language':
            train_data[name] = train_data[name].str.split('~')
            print name
            dummy_names.append(name)

genres
original_language
production_companies
production_countries
spoken_languages


In [6]:
# for each column that was split on '~' except for genre make dummy variables for each
# unique value and concatenate to original data frame
for name in dummy_names:
    if name == 'genres':
        temp = train_data['genres'].str.join(sep='*').str.get_dummies(sep='*')
        labels = ['Unlabeled']*temp.shape[0]
        for i in range(temp.shape[0]):
            if np.sum(temp.loc[i,['28','12','14','878','53','37']]) != 0:
                labels[i] = 'Action'
            if np.sum(temp.loc[i,['80','18','9648','10402','36','10752']]) != 0:
                labels[i] = 'Drama'
            if np.sum(temp.loc[i,['35']]) != 0:
                labels[i] = 'Comedy'
            if np.sum(temp.loc[i,['10751','16']]) != 0:
                labels[i] = 'Family'
            if np.sum(temp.loc[i,['10749']]) != 0:
                labels[i] = 'Romance'
            if np.sum(temp.loc[i,['99']]) != 0:
                labels[i] = 'Documentary'
            if np.sum(temp.loc[i,['27']]) != 0:
                labels[i] = 'Horror'
            if i%15000 == 0:
                print i
        
        labels = pd.DataFrame(labels)
        labels.columns = ['Labels']
        

0
15000
30000
45000
60000
75000
90000
105000
120000
135000
150000


In [7]:
# put all companies in an array to see how many unique companies there are
companies = []
for i in range(train_data.shape[0]):
    companies = np.append(companies, np.array(train_data['production_companies'][i]))
    if i%15000 == 0:
        print i

0
15000
30000
45000
60000
75000
90000
105000
120000
135000
150000


In [30]:
# find unique companies and counts
unique, counts = np.unique(companies, return_counts = True)
len(unique)

46133

In [29]:
# sort by most frequent company
sorted(zip(counts,unique), reverse=True)

[(67438, u'nan'),
 (1476, u'6194'),
 (1325, u'8411'),
 (1074, u'33'),
 (1012, u'4'),
 (894, u'306'),
 (867, u'441'),
 (594, u'5358'),
 (581, u'5798'),
 (569, u'5996'),
 (524, u'5'),
 (501, u'5120'),
 (395, u'3166'),
 (369, u'8520'),
 (366, u'6'),
 (338, u'882'),
 (331, u'3324'),
 (325, u'60'),
 (314, u'1432'),
 (312, u'4395'),
 (295, u'12'),
 (294, u'602'),
 (294, u'2'),
 (260, u'10417'),
 (258, u'4606'),
 (254, u'14317'),
 (244, u'8356'),
 (236, u'9255'),
 (229, u'9195'),
 (229, u'8402'),
 (222, u'18367'),
 (219, u'10330'),
 (216, u'17027'),
 (211, u'955'),
 (206, u'559'),
 (190, u'19788'),
 (188, u'1931'),
 (183, u'7025'),
 (183, u'14'),
 (183, u'10845'),
 (181, u'16466'),
 (179, u'9'),
 (177, u'25'),
 (174, u'235'),
 (169, u'22221'),
 (169, u'18659'),
 (165, u'3823'),
 (163, u'5940'),
 (159, u'6181'),
 (151, u'61'),
 (151, u'288'),
 (149, u'694'),
 (144, u'83'),
 (144, u'5542'),
 (141, u'5766'),
 (137, u'21404'),
 (136, u'328'),
 (133, u'2683'),
 (133, u'1314'),
 (129, u'168'),
 (12

In [37]:
# dummy variables for categories excpet genre and companies
for name in dummy_names:
    if name != 'production_companies' and name != 'genres':
        temp = train_data[name].str.join(sep='*').str.get_dummies(sep='*')
        temp.columns = [name + '_' + str(col) for col in temp.columns]
        train_data = pd.concat([train_data.reset_index(drop=True), temp], axis=1)
        print name

original_language
production_countries
spoken_languages


In [38]:
train_data.shape

(156327, 525)

In [39]:
# remove from features data
train_data = train_data.drop('genres', axis=1)
train_data = train_data.drop('original_language', axis=1)
train_data = train_data.drop('production_countries', axis=1)
train_data = train_data.drop('spoken_languages', axis=1)

In [40]:
train_data.shape

(156327, 521)

In [42]:
features = train_data

In [43]:
features.to_csv('features_V1.csv', index=False, encoding='utf-8')
labels.to_csv('labels.csv', index=False, encoding='utf-8')